<a href="https://colab.research.google.com/github/cnhzgb/MachineL/blob/main/RESNET_TINY_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb
!pip install einops
import ipdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from einops import rearrange
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00


In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) # transforms.RandomHorizontalFlip(), transforms.RandomGrayscale()
dataset = datasets.CIFAR10(root="dataset/", transform=trans, download=True, train=True) # 5W张图片, 10种分类
loader = DataLoader(dataset, batch_size=100, shuffle=True)
batch_num,(image, label) = next(enumerate(loader))
print(len(dataset.classes), len(dataset), image.shape, label.shape) # 10; 5W; 100,3,32,32; 100

trans_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset_test = datasets.CIFAR10(root="dataset/", transform=trans_test, download=True, train=False) # 1W张图片
loader_test = DataLoader(dataset_test, batch_size=100, shuffle=True)
criterion_test = nn.CrossEntropyLoss()

100%|██████████| 170498071/170498071 [00:02<00:00, 76414254.89it/s]


Extracting dataset/cifar-10-python.tar.gz to dataset/
10 50000 torch.Size([100, 3, 32, 32]) torch.Size([100])
Files already downloaded and verified


In [5]:
# https://zhuanlan.zhihu.com/p/87555358
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv11 = nn.Conv2d(3,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv21 = nn.Conv2d(64,128,3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv31 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv32 = nn.Conv2d(256, 256, 3,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv41 = nn.Conv2d(256, 512, 3,padding=1)
        self.conv42 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv51 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv52 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(512*2*2,1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,10)


    def forward(self,x):
        x = F.relu(self.conv11(x)) # => ,64,32,32
        x = self.pool1(x) # => ,64,16,16

        x = F.relu(self.conv21(x)) # => ,128,16,16
        x = self.pool2(x) # => ,128,8,8

        x = F.relu(self.conv31(x)) # => ,256,8,8
        x = F.relu(self.conv32(x))
        x = self.pool3(x) # => ,256,4,4

        x = F.relu(self.conv41(x)) # => ,512,4,4
        x = F.relu(self.conv42(x))
        x = self.pool4(x) # => ,512,2,2

        #x = F.relu(self.conv51(x)) # => ,512,2,2
        #x = F.relu(self.conv52(x))
        # x = self.pool5(x) # => ,512,1,1

        x = x.view(-1,512*2*2)
        x = F.relu(self.fc1(x)) # => ,1024
        x = F.relu(self.fc2(x)) # => ,1024
        x = self.fc3(x) # => ,10
        x = F.softmax(x, dim=1)

        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
         MaxPool2d-2           [-1, 64, 16, 16]               0
            Conv2d-3          [-1, 128, 16, 16]          73,856
         MaxPool2d-4            [-1, 128, 8, 8]               0
            Conv2d-5            [-1, 256, 8, 8]         295,168
            Conv2d-6            [-1, 256, 8, 8]         590,080
         MaxPool2d-7            [-1, 256, 4, 4]               0
            Conv2d-8            [-1, 512, 4, 4]       1,180,160
            Conv2d-9            [-1, 512, 4, 4]       2,359,808
        MaxPool2d-10            [-1, 512, 2, 2]               0
           Linear-11                 [-1, 1024]       2,098,176
           Linear-12                 [-1, 1024]       1,049,600
           Linear-13                   [-1, 10]          10,250
Total params: 7,658,890
Trainable param

In [ ]:
for epoch in range(1,10):
  total_loss = []
  errorTotal = 0
  for batch_idx, (img, label) in enumerate(loader):
    img = img.to(device)
    label = label.to(device)
    outputs = model(img)
    loss = criterion(outputs, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss.append(loss.item())
    maxV,maxIdx = outputs.max(dim=1)
    errorNum = torch.sum(torch.ne(maxIdx, label)).item()
    errorTotal += errorNum

    if(batch_idx % 100 == 99):
      print("epoch:{} batch:{} loss:{:.2f} mean:{:.2f} error:{}/100 errorTotal:{}/{} {:.2f}%".format(
          epoch, batch_idx, loss, np.mean(total_loss), errorNum, errorTotal, (batch_idx+1)*100, errorTotal/(batch_idx+1)))

epoch:1 batch:99 loss:2.18 mean:2.25 error:73/100 errorTotal:8162/10000 81.62%
epoch:1 batch:199 loss:2.16 mean:2.23 error:70/100 errorTotal:15743/20000 78.72%
epoch:1 batch:299 loss:2.16 mean:2.20 error:69/100 errorTotal:22873/30000 76.24%
epoch:1 batch:399 loss:2.10 mean:2.19 error:62/100 errorTotal:29699/40000 74.25%
epoch:1 batch:499 loss:2.03 mean:2.17 error:59/100 errorTotal:36241/50000 72.48%
epoch:2 batch:99 loss:2.01 mean:2.08 error:57/100 errorTotal:6294/10000 62.94%
epoch:2 batch:199 loss:2.04 mean:2.08 error:60/100 errorTotal:12525/20000 62.62%
epoch:2 batch:299 loss:2.09 mean:2.07 error:63/100 errorTotal:18538/30000 61.79%
epoch:2 batch:399 loss:2.02 mean:2.07 error:58/100 errorTotal:24612/40000 61.53%
epoch:2 batch:499 loss:2.04 mean:2.06 error:60/100 errorTotal:30545/50000 61.09%
epoch:3 batch:99 loss:1.94 mean:2.03 error:48/100 errorTotal:5732/10000 57.32%
epoch:3 batch:199 loss:2.01 mean:2.02 error:54/100 errorTotal:11300/20000 56.50%
epoch:3 batch:299 loss:2.04 mean:2